In [3]:
import scipy
from scipy.sparse import hstack
import pandas as pd
pd.options.mode.chained_assignment = None

from ipywidgets import FloatProgress
from IPython.display import display
from string import punctuation
from tqdm import tqdm_notebook
from ipywidgets import FloatProgress

import os
import re
import csv

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, auc, roc_auc_score
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

C:\Users\DELL\Anaconda3\envs\tpml\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [6]:
norec = pd.read_csv('notrecommend.csv',low_memory=False)
applist = pd.read_csv('small_steam_id.csv')
rec_train = pd.read_csv('small_pos_review_cleaned.csv',low_memory=False)

In [7]:
norec_train=norec[['product_id','recommended','text']]
norec_train.head()

,product_id,recommended,text
0,10.0,False,"I never liked this game, nor I do now. I remem..."
1,10.0,False,Phew men that's my childhood right here.\nDon'...
2,10.0,False,"You are 14 years old, first time trying to pla..."
3,10.0,False,this is one of the first games i played as a k...
4,20.0,False,i do not recommend this game in 2019 as there ...


In [62]:
rec_train=rec_train[['appid','review']]
rec_train['sentiment'] = 1
print(rec_train.shape)
print(rec_train.dtypes)
rec_train.head()

(46072, 3)
appid         int64
review       object
sentiment     int64
dtype: object


,appid,review,sentiment
0,10,This old cs. Also like to play some times,1
1,10,Classic game,1
2,10,ok,1
3,10,Good old Cs,1
4,10,ALV,1


In [17]:
review_norec=norec_train[norec_train['product_id'].isin(applist['appid'])]
len(review_norec.product_id.unique())

2971

In [23]:
review_norec = review_norec.dropna().reset_index(drop = True)
review = []
for i in range(len(review_norec)):
    s = re.compile(r'[^A-Za-z0-9\s\-\.\/]',re.S)
    if str(review_norec.loc[i]['text']) == 'nan':
        review.append('')
        continue
    try:
        review.append(s.sub('',review_norec.loc[i]['text']))
    except TypeError as er:
        print(review_norec.loc[i]['text'])
        pass
    
review_norec['review'] = review

KeyError: "['recommended\\t'] not in index"

In [36]:
review_norec = review_norec[['product_id','review','recommended']].dropna().reset_index(drop=True)
review_norec.columns = ['appid','review','recommended']
print(review_norec.shape)
print(review_norec.isnull().sum())
print(review_norec['recommended'].value_counts())
print(type(review_norec.loc[0,'recommended']))
review_norec.loc[review_norec['recommended']==True]

False    58713
True         1
Name: recommended, dtype: int64
<class 'bool'>


,appid,review,recommended
44394,469600.0,Its ok.\nI wish they would re-implement the 4v...,True


In [56]:
#review_norec = review_norec.rename(columns={'product_id':'appid'}).astype({'appid':'int64','recommended':'object'})
review_norec = review_norec.loc[review_norec['recommended'] == False]
print(review_norec['recommended'].value_counts())

False    58713
Name: recommended, dtype: int64


In [63]:
review_norec['sentiment'] = 0
review_norec = review_norec[['appid','review','sentiment']]
print(review_norec.shape)
print(review_norec.isnull().sum())
print(review_norec['sentiment'].value_counts())
review_norec.tail()

(58713, 3)
appid         0
review       60
sentiment     0
dtype: int64
0    58713
Name: sentiment, dtype: int64


,appid,review,sentiment
58709,1052070,This a really neat interactive story.\nIt is a...,0
58710,1052070,I mean... its pretty. but outside of that its ...,0
58711,1052070,Heavy bad glitches just no.,0
58712,1052070,A vanity project for a graphic artist and musi...,0
58713,1052070,I really wanted to like this game as it was ma...,0


In [64]:
review_norec.dropna(inplace=True)
print(review_norec.isnull().sum())

appid        0
review       0
sentiment    0
dtype: int64


In [67]:
review_small = pd.concat([review_norec,rec_train],ignore_index=True)
review_small.dropna(inplace=True)
print(review_small.dtypes)
print(review_small['sentiment'].value_counts())
print(review_small.shape)
print(review_small.isnull().sum())
display(review_small.tail())

appid         int64
review       object
sentiment     int64
dtype: object
0    58653
1    46072
Name: sentiment, dtype: int64
(104725, 3)
appid        0
review       0
sentiment    0
dtype: int64


,appid,review,sentiment
104720,1052070,Its basically a walking simulator with limited...,1
104721,1052070,This is not a game it is a piece of artistic e...,1
104722,1052070,Incredible game. It takes place in a cyberpunk...,1
104723,1052070,One of the best free game on Steam. Superb vis...,1
104724,1052070,Extremely interesting art game by 12 students ...,1


# Data Process

In [72]:
x_train, x_test, y_train, y_test = train_test_split(pd.DataFrame(review_small[['review','sentiment']]), 
                                                    pd.Series(review_small['sentiment']),
                                                    test_size=0.2, 
                                                    random_state=0)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(83780, 2) (20945, 2) (83780,) (20945,)


In [74]:
import collections
SteamDocument = collections.namedtuple('SteamDocument','words tags split sentiment')

In [75]:
import gensim.utils
Steam_train = []
for i in range(len(x_train)):
    tokens = gensim.utils.simple_preprocess(x_train.iloc[i][0])
    Steam_train.append(SteamDocument(tokens, [i], 'train', x_train.iloc[i][1]))
print(len(Steam_train))
print(Steam_train[10])

83780
SteamDocument(words=['it', 'crashes', 'upon', 'start'], tags=[1], split='train', sentiment=0)


In [77]:
Steam_test = []
for i in range(len(x_test)):
    tokens = gensim.utils.simple_preprocess(x_test.iloc[i][0])
    Steam_test.append(SteamDocument(tokens, [i], 'test', x_test.iloc[i][1]))
print(len(Steam_test))
print(Steam_test[1])

20945
SteamDocument(words=['finished', 'on', 'april', 'automatic', 'save', 'combat', 'mechanism', 'except', 'melee', 'graphics', 'weapons', 'can', 'be', 'upgraded', 'if', 'the', 'same', 'weapon', 'is', 'picked', 'on', 'the', 'field', 'characters', 'dialogs', 'quality', 'level', 'design', 'no', 'extra', 'objectives', 'side', 'quests', 'story', 'voice', 'acting'], tags=[1], split='test', sentiment=0)


In [78]:
alldocs = Steam_train + Steam_test
len(alldocs)

104725

# Define Model

In [79]:
import multiprocessing
from collections import OrderedDict

import gensim.models.doc2vec
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

from gensim.models.doc2vec import Doc2Vec

In [80]:
common_kwargs = dict(vector_size = 100, epochs = 40, min_count = 2,
                    sample = 0, workers = multiprocessing.cpu_count(),
                    negative = 5, hs = 0,)

simple_models = [
    #PV-DBOW
    Doc2Vec(dm=0, **common_kwargs),
    #PV-DM /average
    Doc2Vec(dm=1, window=10, alpha=0.05, comment='alpha=0.05', **common_kwargs),
    #PV-DM /concatenation
    Doc2Vec(dm=1, dm_concat = 1, window = 5, **common_kwargs),
]

for model in simple_models:
    model.build_vocab(alldocs)
    print("%s vocabulary built and state initialized" % model)

Doc2Vec(dbow,d100,n5,mc2,t12) vocabulary built and state initialized
Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t12) vocabulary built and state initialized
Doc2Vec(dm/c,d100,n5,w5,mc2,t12) vocabulary built and state initialized


In [81]:
models_by_name = OrderedDict((str(model),model) for model in simple_models)
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[1]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[2]])

# Define evaluate function

In [ ]:
import numpy as np
#import statsmodels.api as sm
from random import sample
from sklearn.metrics import precision_recall_fscore_support
#from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
def svc_predictor_from_data(train_targets, train_regressors):
    """Fit a svc predictor on supplied data"""
    clf = SVC(kernel='rbf',C=0.4,class_weight='balanced')
    clf.fit(train_regressors, train_targets)
#     logit = sm.Logit(train_targets, train_regressors)
#     predictor = logit.fit(disp=0)
    # print(predictor.summary())
    return clf

def error_rate_for_model(test_model, train_set, test_set, test_target):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets = [doc.sentiment for doc in train_set]
    train_regressors = [test_model.docvecs[doc.tags[0]] for doc in train_set]
    #train_regressors = sm.add_constant(train_regressors)
    predictor = svc_predictor_from_data(train_targets, train_regressors)

    test_regressors = [test_model.docvecs[doc.tags[0]] for doc in test_set]
    #test_regressors = sm.add_constant(test_regressors)

    # Predict & evaluate
    pre_y = predictor.predict(test_regressors)
    accuracy = accuracy_score(test_target, pre_y)
    precision,recall,f1,support = precision_recall_fscore_support(test_target, pre_y)
    return (accuracy, precision,recall,f1)

# Test

In [86]:
%%time
from random import shuffle
shuffled_alldocs = alldocs[:]
shuffle(shuffled_alldocs)
for model in simple_models:
    print("Training %s" % model)
    model.train(shuffled_alldocs, total_examples=len(shuffled_alldocs), epochs=model.epochs)

    print("\nEvaluating %s" % model)
    accuracy, precision,recall,f1 = error_rate_for_model(model, Steam_train, Steam_test,y_test)
    #error_rates[str(model)] = err_rate
    print('accuracy:',accuracy,'\nprecision:',precision,'\nrecall:',recall,'\nF1:',f1)
    print('='*60)

for model in [models_by_name['dbow+dmm'], models_by_name['dbow+dmc']]:
    print("\nEvaluating %s" % model)
    accuracy, precision,recall,f1 = error_rate_for_model(model, Steam_train, Steam_test, y_test)
    #error_rates[str(model)] = err_rate
    print('accuracy:',accuracy,'\nprecision:',precision,'\nrecall:',recall,'\nF1:',f1)
    print('='*60)

Training Doc2Vec(dbow,d100,n5,mc2,t12)

Evaluating Doc2Vec(dbow,d100,n5,mc2,t12)
accuracy: 0.6863213177369301 
precision: [0.71303775 0.649529  ] 
recall: [0.73696763 0.62172732] 
F1: [0.72480523 0.63532416]
Training Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t12)

Evaluating Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t12)
accuracy: 0.6615898782525662 
precision: [0.69200925 0.61986185] 
recall: [0.71405451 0.59467681] 
F1: [0.70285906 0.60700821]
Training Doc2Vec(dm/c,d100,n5,w5,mc2,t12)

Evaluating Doc2Vec(dm/c,d100,n5,w5,mc2,t12)
accuracy: 0.6299355454762473 
precision: [0.66161575 0.58449558] 
recall: [0.69548552 0.54633351] 
F1: [0.67812798 0.56477062]

Evaluating Doc2Vec(dbow,d100,n5,mc2,t12)+Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t12)
accuracy: 0.6692289329195512 
precision: [0.69516548 0.63212255] 
recall: [0.72998296 0.59174362] 
F1: [0.71214891 0.61126697]

Evaluating Doc2Vec(dbow,d100,n5,mc2,t12)+Doc2Vec(dm/c,d100,n5,w5,mc2,t12)
accuracy: 0.6793029362616376 
precision: [0.7